In [3]:
"""
Created on Wed Jul  7 17:46:12 2021

@author: Schroeder
"""

from turtle import color
import matplotlib
from matplotlib import colors

from IPython.display import display, HTML

from pathlib import Path
import csv

from datetime import datetime
from datetime import date
import calendar

import os
from os import listdir
from os.path import isfile, join

import sys, getopt

# Imports
#from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
import numpy as np
from sklearn import linear_model
import scipy.stats


import numpy as np
from sklearn import linear_model

import pandas as pd

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

from pandas import ExcelWriter

from sympy import maximum

import RSI_strat_SETUP

#init_notebook_mode()

In [4]:
def main(fad, stock):


    #onlyfiles = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and not f.startswith("_")) ]

    b=[]
    c=[]
    figs=[]
    counter  = 0

    keys = ["Sector","Industry"]
    a=[]    
    b=[]
    
    for key in keys:
        'Baue Dataframe aus der exisitierendnen Res Datei. Relevnt sind die Sektoren und die 5d,10,20d,... returns'
        df = pd.read_csv(fad + stock,sep=";",decimal=',', index_col=0)
        csa = [key]
        csa = csa +  df.columns[:5].tolist()
        sector_analysis = pd.DataFrame(columns=csa)

        list_of_sectors = df[key].unique().tolist()
        sector_analysis[key]=df[key].unique().tolist()

        # Wenn nach Industry aufgelöst wird, dann fuerg den Sektor in den TAble dazu 
        if key == "Industry":
            print(sector_analysis["Industry"])
            for i in sector_analysis["Industry"]:
                a=df["Sector"][df["Industry"]==i]
                b.append(a[0])    
            print(b)    
            sector_analysis["Sector"]=b
        ################################

        for dt in df.columns[:5].tolist():
            yields=[]
            for sec in sector_analysis[key]:
                yields.append(df.loc[df[key]==sec, dt].mean())
            #print(yields)
            sector_analysis[dt]=yields

        sector_analysis=sector_analysis.sort_values(by="10d",ascending=False)
        sector_analysis.style.background_gradient()        

        f = open(fad+key+"_s.html",'w'); f.write(sector_analysis.style.set_precision(2).set_caption('Spaltenweise Color').background_gradient(axis=0,cmap="Spectral").render()); f.close()
        f = open(fad+key+"_z.html",'w'); f.write(sector_analysis.style.set_precision(2).set_caption('Zeilenweise Color').background_gradient(axis=1,cmap="Spectral").render()); f.close()    
        
    

### Gebe Pfad an, unter dem das Datenfile zu finden ist.  
#### Bsp: C:/Users/Schroeder/OneDrive/Trading/ETFS/RES_Aristokraten/Res/

In [19]:
currentdir = os.path.abspath('')
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
#############################################################

from MomentumScreening import my_setup
###
### The whole /ETFS/ Tree has to be located on ame level as repository !
pfad = os.path.realpath(os.path.join(parentdir, 'ETFS'))


### --------------------------------------------------------------------------
### Welcher Sektor oder index soll analysieert werden ?

In [20]:
who = "SP500"


who = "RES_" + who 
pfad = os.path.realpath(os.path.join(pfad, who))
pfad = os.path.realpath(os.path.join(pfad, 'Res'))

file  = "SP502022-07-29__2022-04-06" + ".csv"

### ---------------------------------------------------------------------

In [21]:

print(pfad)
print(pfad + "\\" + file)

C:\Users\Michael\Trading\ETFS\RES_SP500\Res
C:\Users\Michael\Trading\ETFS\RES_SP500\Res\SP502022-07-29__2022-04-06.csv


In [8]:
#main(stock,          back_gap, fwd_gap, threshold rendite backward, threshold renidte forward,comment )

main(pfad, file)

0                           Personal Services
1                     Beverages—Non-Alcoholic
2      Drug Manufacturers—Specialty & Generic
3                              Packaged Foods
4                  Drug Manufacturers—General
                        ...                  
111                                       ETF
112                            Grocery Stores
113                                     Steel
114                                    Copper
115                                      Gold
Name: Industry, Length: 116, dtype: object
['Consumer Cyclical', 'Consumer Defensive', 'Healthcare', 'Consumer Defensive', 'Healthcare', 'Healthcare', 'Industrials', 'Healthcare', 'Technology', 'Real Estate', 'Consumer Cyclical', 'Financial Services', 'Consumer Defensive', 'Real Estate', 'Technology', 'Industrials', 'Healthcare', 'Consumer Defensive', 'Communication Services', 'Consumer Cyclical', 'Healthcare', 'Consumer Defensive', 'Consumer Cyclical', 'Utilities', 'Consumer Defensive', '